# Končni projekt

Izdelali bomo komplicirano aplikacijo za izračun fibbocacijevega zaporedja. 

- Multikontejnerska aplikacija
- Ni pomembna logika apliakicje

## Arhitektura

- NGINX, React Server, Express Server, Postgress, Redis, Worker

## Komponente

### Worker

Mapa `worker`

- `index.js` -> logika (počasna verzija, da lažje prikažemo obremenitve)
- `keys.js` -> preberemo podatke za se povezat na redis
- `package.json`

#### Dev Dockerfiles

- naredimo Dockerfile.dev
- Build: `docker build -f Dockerfile.dev .` 
- Zaženemo: `docker run 8e0aa51ba18a`

#### Dockerfiles

Dodamo Dockerfile

### API Server (Express server)

Mapa `server`

- `index.js` -> logika
    - povezava na redis
    - povezava na postgres (shranjuje že vstavljene vrednosti)
    - odgovor na API klice
- `keys.js` -> preberemo podatke za se povezat na redis, postgres
- `package.json`

#### Dev Dockerfiles

- naredimo Dockerfile.dev
- Build: `docker build -f Dockerfile.dev .` 
- Zaženemo: `docker run 0847fc503335`

#### Dockerfiles

Dodamo Dockerfile

### Frontend (React App)

Mapa `client`

Frontend za naš projekt

#### Dev Dockerfiles

- naredimo Dockerfile.dev
- Build: `docker build -f Dockerfile.dev .` 
- Zaženemo: `docker run -p 3000:3000 b88389f464f2`

#### Dockerfiles

Dodamo Dockerfile

    docker build -t leon11sj/multi-client:v1 .
    docker push leon11sj/multi-client:v1

## docker-compose za vse datoteke

Ustvarimo `docker-compose.yml` za lokalni test vseh storitev.

Dodajamo po vrsti:
- postgres
- redis
- server
- worker & client

Dodati moramo še NGINX, da bomo lahko dostopali do storitev. -> Določa nam pathe za dostop do določene storitve.
- kar se začne `/api/` -> presumerimo na express server
- ostalo na react server

Nardimo mapo `nginx`
- default.conf
- Dockerfile.dev
- Dockerfile

V produkcijski verziji odstranimo del z socketi v nginx config.

## Kubernetes

### client-deployment

```yml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: client-deployment
spec:
  replicas: 3
  selector:
    matchLabels:
      component: web
  template:
    metadata:
      labels:
        component: web
    spec:
      containers:
        - name: client
          image: leon11sj/multi-client:v1
          ports:
            - containerPort: 3000
```

    kubectl apply -f client-deployment.yaml

### client-cluster-ip-service

```yml
apiVersion: v1
kind: Service
metadata:
  name: client-cluster-ip-service
spec:
  type: ClusterIP
  selector:
    component: web
  ports:
    - port: 3000
      targetPort: 3000

```

    kubectl apply -f client-cluster-ip-service.yaml

### redis-deployment



```yml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: redis-deployment
spec:
  replicas: 1
  selector:
    matchLabels:
      component: redis
  template:
    metadata:
      labels:
        component: redis
    spec:
      containers:
        - name: redis
          image: redis
          ports:
            - containerPort: 6379
```

    kubectl apply -f redis-deployment.yaml

### redis-cluster-ip-service

```yml
apiVersion: v1
kind: Service
metadata:
  name: redis-cluster-ip-service
spec:
  type: ClusterIP
  selector:
    component: redis
  ports:
    - port: 6379
      targetPort: 6379

```

    kubectl apply -f redis-cluster-ip-service.yaml

In [ ]:
pvc

In [ ]:
secret

### postgres-deployment.yaml

```yml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: postgres-deployment
spec:
  replicas: 1
  selector:
    matchLabels:
      component: postgres
  template:
    metadata:
      labels:
        component: postgres
    spec:
      volumes:
        - name: postgres-storage
          persistentVolumeClaim:
            claimName: database-persistent-volume-claim
      containers:
        - name: postgres
          image: postgres
          ports:
            - containerPort: 5432
          volumeMounts:
            - name: postgres-storage
              mountPath: /var/lib/postgresql/data
              subPath: postgres
          env:
            - name: PGPASSWORD
              valueFrom:
                secretKeyRef:
                  name: pgpassword
                  key: PGPASSWORD
```

### postgres-cluster-ip-service

```yml
apiVersion: v1
kind: Service
metadata:
  name: postgres-cluster-ip-service
spec:
  type: ClusterIP
  selector:
    component: postgres
  ports:
    - port: 5432
      targetPort: 5432
```

### server-deployment

```yml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: server-deployment
spec:
  replicas: 3
  selector:
    matchLabels:
      component: server
  template:
    metadata:
      labels:
        component: server
    spec:
      containers:
        - name: server
          image: stephengrider/multi-server
          ports:
            - containerPort: 5000
          env:
            - name: REDIS_HOST
              value: redis-cluster-ip-service
            - name: REDIS_PORT
              value: '6379'
            - name: PGUSER
              value: postgres
            - name: PGHOST
              value: postgres-cluster-ip-service
            - name: PGPORT
              value: '5432'
            - name: PGDATABASE
              value: postgres
            - name: PGPASSWORD
              valueFrom:
                secretKeyRef:
                  name: pgpassword
                  key: PGPASSWORD
                    
```

### server-cluster-ip-service

```yml
apiVersion: v1
kind: Service
metadata:
  name: server-cluster-ip-service
spec:
  type: ClusterIP
  selector:
    component: server
  ports:
    - port: 5000
      targetPort: 5000
```

### worker-deployment

```yml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: worker-deployment
spec:
  replicas: 1
  selector:
    matchLabels:
      component: worker
  template:
    metadata:
      labels:
        component: worker
    spec:
      containers:
        - name: worker
          image: stephengrider/multi-worker
          env:
            - name: REDIS_HOST
              value: redis-cluster-ip-service
            - name: REDIS_PORT
              value: '6379'
```